<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D1780_Check_if_Number_is_a_Sum_of_Powers_of_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1780. Check if Number is a Sum of Powers of Three

https://leetcode.com/problems/check-if-number-is-a-sum-of-powers-of-three/description/

Given an integer `n`, return `true` if it is possible to represent `n` as the sum of distinct powers of three. Otherwise, return `false`.

An integer `y` is a power of three if there exists an integer `x` such that `y == 3`<sup>`x`</sup>.

**Constraints:**
- `1 <= n <= 10`<sup>`7`</sup>

**Example 1:**

- Input: `n = 12`

- Output: `true`

- Explanation: `12 = 3`<sup>`1`</sup>` + 3`<sup>`2`</sup>


**Example 2:**

- Input: `n = 91`

- Output: `true`

- Explanation: `91 = 3`<sup>`0`</sup>` + 3`<sup>`2`</sup>` + 3`<sup>`4`</sup>.

**Example 3:**

- Input: `n = 21`

- Output: `false`

In [2]:
test_cases = [
    {
        "input": {
            "n": 12,
        },
        "output": True
    },
    {
        "input": {
            "n": 91,
        },
        "output": True
    },

    {
        "input": {
            "n": 21,
        },
        "output": False
    },
]

### Observations

- If `n` is the sum of distinct powers of `3`, then `n modulo 3 in (0, 1)`

  - If `n modulo 3 = 2` then `n` is not sum of power of `3`

  - If `n modulo 3 = 1`, then `3`<sup>`0`</sup> must be included for `n` is satified the conditions.

- Pre compute all power of `3` which smaller than `n`

- Recursively try all possible combination of power of `3`, start from largest to smallest

### Analysis

- Time to compute `powers` array: `log`<sub>`3`</sub>`n`

- First recursive round: `log`<sub>`3`</sub>`n`

  - Second recursive round: `log`<sub>`3`</sub>`n - 1`

  - Third recursive round: `log`<sub>`3`</sub>`n - 2`

  ...

  - Last recurive round: `1`

- Time complexity:
    
    `log`<sub>`3`</sub>`n` + `log`<sub>`3`</sub>`n` + `log`<sub>`3`</sub>`n - 1` + `log`<sub>`3`</sub>`n - 2` + ...

    = `O((log`<sub>`3`</sub>`n)`<sup>`2`</sup>`)`

### More over

- Better approach:

  - If n is greater than or equal to the current power, skipping this power will always lead to a false result.
    
  - This is because the largest sum we can achieve with smaller powers is the sum of all lower powers, which is always less than the current power.
  
  - So, if we skip this power, we can’t form the sum n, and we must include it by subtracting it from n.

  - Useful formula: let `3`<sup>`x`</sup> is the largest in all powers of 3 that smaller than `n`, `3`<sup>`x`</sup>` <= n`

   $3^0 + 3^1 + 3^2 + ... + 3^{x-1} = \frac{3^x - 1}{2} < 3^x <= n$

  - Let $3^x$ is the largest power of `3` that smaller than `n`, then $3^x$ must be included in the list of power of `3` that make up to `n`

    - Subtract `n` by $3^x$

    - Continue find the next $3^x$ that nearest to `n`

    - Repeat until `n = 0`, retrun `True`

    - Otherwise return `False`

### Implementation

In [28]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue

class Solution:
    def checkPowersOfThree(self, n: int) -> bool:
        # pre-compute list of distinct power of 3
        powers = []
        power = 0
        power_of_three = 3 ** power
        while power_of_three < n:
            powers.append(power_of_three)
            power += 1
            power_of_three = 3 ** power

        if power_of_three == n:
            return True

        # pre-check
        if n % 3 == 2:
            return False

        return self.isPowerOfThree(n, powers[::-1], set())

    def isPowerOfThree(self, remaning, powers, processed_power) -> bool:
        # base cases
        if remaning == 0:
            return True

        if remaning < 0:
            return False

        for power in powers:
            if power <= remaning and power not in processed_power:
                processed_power.add(power)
                return self.isPowerOfThree(remaning - power, powers, processed_power)

        return False

In [27]:
solution = Solution()
for test_case in test_cases:
    print()
    result = solution.checkPowersOfThree(test_case["input"]["n"])
    if result != test_case["output"]:
        print(f'Failed test case with input: {test_case["input"]} and output: {test_case["output"]} vs result: {result}')

## Leetcode Editorial Approach 3: Ternary Representation
### Intuition

- First, let's break the problem down into a more familiar one. We know that every number can be written as a sum of distinct powers of `2` — in other words, every number has a **unique** binary representation. A simple way to find the binary representation of a number is by repeatedly taking its remainder when divided by `2 (mod 2)` and then dividing the number by `2` to move to the next bit. This method is similar to the two’s complement approach.

- For example:

 $14 = 0 * 2^3 + 1 * 2^2 + 1 * 2^1 + 1 * 2^0 $ -> binary representation `base 2` $0111$

 $31 = 1 * 3^3 + 0 * 3^2 + 1 * 3^1 + 1 * 3^0 $ -> binary representation `base 3` $1011$

- If `n` is the sum of distinct powers of `3`, then `n` must be able to represent as binary with `base 3` (a.k.a Ternary Representation)

- In this problem, we apply the same logic but in base `3` instead of base `2`. We construct the ternary representation of the given number by taking its remainder when divided by `3 (mod 3)` and then dividing it by `3` to proceed to the next digit. If any of these remainders equals `2`, we would need to use a power of `3` twice, which is not allowed. In that case, we immediately return `false`.



In [1]:
class Solution:
    def checkPowersOfThree(self, n: int) -> bool:
        while n > 0:
            # if n mod 3 remainer is not 1 or 0
            if n % 3 == 2:
                return False
            n = n // 3

        return True